<a href="https://colab.research.google.com/github/aryanchaprana/aryanchaprana/blob/main/ML_7(SCIKIT_LEARN_PIPELINES)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import pandas as pd

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [59]:
df=pd.read_csv('/content/train (2).csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [61]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [62]:




X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [78]:
nulll=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')



In [64]:
onehot=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')#sex aur embarked pai



In [65]:
minmax=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) #slice matlab ki range jaise hmne 0 se 9 liya h 10 exclude hojata hai
])



In [79]:
feature_selection=SelectKBest(chi2,k=8)


In [66]:
tree=DecisionTreeClassifier()

### pipeline vs make_pipeline

In [82]:
pipe=Pipeline([
    ('nulll',nulll),
    ('onehot',onehot),
    ('minmax',minmax),
    ('feature_selection',feature_selection),
    ('tree',tree)
])



In [73]:
pipe=make_pipeline(nulll,onehot,minmax,tree) #easier syntax than pipeline

In [81]:
pipe.fit(X_train,y_train)

pipe.fit(X_train,y_train)

Pipeline(steps=[('nulll',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('onehot',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('minmax',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('feature_selection',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7b4e44ccc790>)),
                ('tree', DecisionTreeClassifier())])

In [83]:
y_pred=pipe.predict(X_test)

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

exporting pipeline


In [85]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
test_input=np.array(['2','male','20','1','0','80','C'],dtype=object).reshape(1,7)
pipe.predict(test_input)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])